In [ ]:
# %env MINERL_DATA_ROOT=/Volumes/CORSAIR/data

In [ ]:
import minerl  # NOTE: we need gym>=0.13.1,<0.20
import gym
from train import get_agent, get_dynamics_environment, FMC, get_data_handler
from fgz.training.fgz_trainer import FGZTrainer
import torch
import wandb

In [ ]:
torch.cuda.empty_cache()  # fix memory leaks

In [ ]:
minerl_env = gym.make('MineRLBasaltBuildVillageHouse-v0')
agent = get_agent()
dynamics_env = get_dynamics_environment(minerl_env)

In [ ]:
agent.device

In [ ]:
# dynamics_env.batched_action_space_sample()

In [ ]:
# dummy_initial_state = torch.ones(4096, dtype=float)
# dynamics_env.set_all_states(dummy_initial_state)

In [ ]:
data_handler = get_data_handler(agent)
str(data_handler.loaders[0].trajectories[0])

In [ ]:
# t = data_handler.sample_single_trajectory()
# for window in t:
#     print(len(window))
#     for frame, state_embedding, action in window:
#         print(state_embedding.shape)
#         break
#     break

In [ ]:
dynamics_function_optimizer = torch.optim.Adam(
    dynamics_env.dynamics_function.parameters(),
    lr=0.00002,
    weight_decay=1e-4,
)

In [ ]:
use_wandb = True

fmc = FMC(dynamics_env, freeze_best=True)
trainer = FGZTrainer(agent, fmc, data_handler, dynamics_function_optimizer, unroll_steps=8, use_wandb=use_wandb)

if use_wandb:
    wandb.init(project="fgz-v0")

In [ ]:
# trainer.save("test_trainer_save.pth")
# loaded_trainer = FGZTrainer.load("test_trainer_save.pth", agent)
# loaded_trainer.evaluate("MineRLBasaltMakeWaterfall-v0", render=True, max_steps=4096, force_no_escape=True)

In [ ]:
# trainer.evaluate("MineRLBasaltMakeWaterfall-v0", render=True, max_steps=4096, force_no_escape=True)

In [ ]:
# trainer.eval_actions

In [ ]:
train_steps = 1000
for _ in range(train_steps):
    trainer.train_sub_trajectory(use_tqdm=True)

In [ ]:
trainer.save("fgz_dynamics_trained.pth")

In [ ]:
trainer = FGZTrainer.load("fgz_dynamics_trained.pth", agent)

In [ ]:
trainer.evaluate("MineRLBasaltMakeWaterfall-v0", render=True, max_steps=4096, force_no_escape=True)